## Feistel Cipher  

A Feistel cipher is a symmetric structure used in the construction of block ciphers, named after the German-born physicist and cryptographer Horst Feistel. A large proportion of block ciphers use the scheme, including the Data Encryption Standard (DES). The Feistel structure has the advantage that encryption and decryption operations are very similar, even identical in some cases, requiring only a reversal of the key schedule.  

The Feistel cipher consists of multiple rounds of processing of the plaintext, each round consisting of a “substitution” step followed by a permutation step.  


- The input block to each round is divided into two halves that can be denoted as L and R for the left half and the right half.
- In each round, the right half of the block, R, goes through unchanged. But the left half, L, goes through an operation that depends on R and the encryption key. First, we apply an encrypting function ‘f’ that takes two input − the key K and R. The function produces the output f(R,K). Then, we XOR the output of the mathematical function with L.
- In real implementation of the Feistel Cipher, such as DES, instead of using the whole encryption key during each round, a round-dependent key (a subkey) is derived from the encryption key. This means that each round uses a different key, although all these subkeys are related to the original key.
- The permutation step at the end of each round swaps the modified L and unmodified R. Therefore, the L for the next round would be R of the current round. And R for the next round be the output L of the current round.
- Above substitution and permutation steps form a ‘round’. The number of rounds are specified by the algorithm design.
- Once the last round is completed then the two sub blocks, ‘R’ and ‘L’ are concatenated in this order to form the ciphertext block.


<img src="images/feistel.jpg" style="height:350px">

Source: https://engineering.purdue.edu/kak/compsec/NewLectures/Lecture3.pdf  
Code adapted from: https://github.com/fgutica/COMP7401-Feistel-Cipher

In [1]:
import hashlib

In [2]:
class Feistel():
    
    def __init__(self, key, mode):
        self.ROUNDS = 8
        self.BLOCKSIZE = 8
        self.BLOCKSIZE_BITS = 64
        self.SECRET = key
        self.mode = mode

    def key_256(self, key):
        return hashlib.sha256((key + self.SECRET).encode('utf-8')).hexdigest()

    def subkeygen(self, s1, s2, i):
        result = hashlib.sha256((s1 + s2).encode('utf-8')).hexdigest()
        return result

    def xor(self, s1, s2):
        return ''.join(chr(ord(a) ^ ord(b)) for a, b in zip(s1, s2))

    def scramble(self, x, i, k):
        k = self.stobin(k)
        x = self.stobin(str(x))

        k = self.bintoint(k)
        x = self.bintoint(x)

        res = pow((x * k), i)
        res = self.itobin(res)

        return self.bintostr(res)

    # string to binary
    def stobin(self, s):
        return ''.join('{:08b}'.format(ord(c)) for c in s)

    # binary to int
    def bintoint(self, s):
        return int(s, 2)

    # int to binary
    def itobin(self, i):
        return bin(i)

    # binary to string
    def bintostr(self, b):
        n = int(b, 2)
        return ''.join(chr(int(b[i: i + 8], 2)) for i in range(0, len(b), 8))

    def encryptMessage(self, message, key, mode):
        """
        Mode: ecb or cbc
        """
        ciphertext = ""
        n = self.BLOCKSIZE  # 8 bytes (64 bits) per block

        # Split message into 64-bit blocks
        message = [message[i: i + n] for i in range(0, len(message), n)]

        lengthOfLastBlock = len(message[len(message)-1])
        if ( lengthOfLastBlock < self.BLOCKSIZE):
            for i in range(lengthOfLastBlock, self.BLOCKSIZE):
                message[len(message)-1] += " "

        # Generate a 256 bit key based of user inputted key
        key = self.key_256(key)
        key_initial = key

        for block in message:
            L = [""] * (self.ROUNDS + 1)
            R = [""] * (self.ROUNDS + 1)
            L[0] = block[0: int(self.BLOCKSIZE/2)]
            R[0] = block[int(self.BLOCKSIZE/2): self.BLOCKSIZE]

            for i in range(1, self.ROUNDS+1):
                L[i] = R[i - 1]

                if (self.mode == "cbc"):
                    if (i == 1):
                        key = key_initial
                    else:
                        key = self.subkeygen(L[i], key_initial, i)

                R[i] = self.xor(L[i - 1], self.scramble(R[i - 1], i, key))

            ciphertext += (L[self.ROUNDS] + R[self.ROUNDS])

        return ciphertext

    def decryptCipher(self, key, ciphertext, mode):
        message = ""
        n = self.BLOCKSIZE  # 8 bytes (64 bits) per block

        # Split message into 64bit blocks
        ciphertext = [ciphertext[i: i + n] for i in range(0, len(ciphertext), n)]

        # Pad last block, if necessary, with spaces
        lengthOfLastBlock = len(ciphertext[len(ciphertext)-1])
        if ( lengthOfLastBlock < self.BLOCKSIZE):
            for i in range(lengthOfLastBlock, self.BLOCKSIZE):
                ciphertext[len(ciphertext)-1] += " "

        # Generate a 256 bit key based off the user inputted key
        key = self.key_256(key)
        key_initial = key
        
        for block in ciphertext:
            L = [""] * (self.ROUNDS + 1)
            R = [""] * (self.ROUNDS + 1)
            L[self.ROUNDS] = block[0:int(self.BLOCKSIZE/2)]
            R[self.ROUNDS] = block[int(self.BLOCKSIZE/2): self.BLOCKSIZE]

            for i in range(8, 0, -1):

                if (self.mode == "cbc"):
                    key = self.subkeygen(L[i], key_initial, i)

                    if (i == 1):
                        key = key_initial

                R[i-1] = L[i]
                L[i-1] = self.xor(R[i], self.scramble(L[i], i, key))

            message += (L[0] + R[0])

        return message

In [3]:
key = "3f788083-77d3-4502-9d71-21319f1792b6"
mode = "cbc"
f = Feistel(key, mode)
message = "Horst Feistel (January 30, 1915 – November 14, 1990) was a German-born cryptographer."
encrypted_text = f.encryptMessage(message, key, mode)
print(encrypted_text)

!»û)‟`åyrt: 6Ã`=¢K38ãz=ä>i¾'üP*bz·Qj¨ddÄ"ÇÙ#<(þ5


In [4]:
decrypted_text = f.decryptCipher(key, encrypted_text, mode)
print(decrypted_text)

Horst Feistel (January 30, 1915 – November 14, 1990) was a German-born cryptographer.   
